# chapter 7 对象应用，可变性和垃圾回收



## 装饰器基础知识

装饰器是可调用的对象，其参数是另一个函数。装饰器可能会处理被装饰的函数，然后把它返回，或者将其替换成另一个函数或可调用对象。

```python
@decorate
def target():
    print('runnning target()')
```

上述代码的效果与下面的一样:

```python
def target():
    print('running target()')
    
target = decorate(target)
```

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [2]:
target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner>

可以看出来被装饰的`target`其实会运行`inner`；

target其实是inner的引用

## Python何时执行装饰器

装饰器的关键特性是，他们在被装饰的函数定义之后立即执行。时常是在导入模块时

```python
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    

@register
def f2():
    print('running f2()')

@register
def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__=='__main__':
    main()
```

In [4]:
!python3 registration.py

running register(<function f1 at 0x1017b6b70>)
running register(<function f2 at 0x1017ba1e0>)
running register(<function f3 at 0x1017ba268>)
running main()
registry -> [<function f1 at 0x1017b6b70>, <function f2 at 0x1017ba1e0>, <function f3 at 0x1017ba268>]
running f1()
running f2()
running f3()


装饰器在导入模块的时候立即执行，被包装的函数在被调用的时候才运行；上述例子其实不常见:

+ 实际情况中，装饰器通常在一个模块中定义，然后应用到其他模块的函数上；
+ register函数返回的函数与通过参数传入的相同；实际中，大多数装饰器会在内部定义一个函数，然后将其返回。

## 使用装饰器改进”策略“模式

In [5]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # 上下文
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
            
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

    


In [6]:
promos = []
# 装饰器
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
            
    return discount

@promotion
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

多数装饰器会修改被装饰的函数，通常它们会定义一个内部函数，然后将其返回，替换被装饰的函数。使用内部函数的代码几乎都要靠闭包才能正确运行。

## 变量作用域规则



In [7]:
b=6
def f2(a):
    print(a)
    print(b)
    b = 3
    
f2(1)

1


UnboundLocalError: local variable 'b' referenced before assignment

Python函数的定义体里面，对于尝试赋值的变量，都认为是局部变量，所以上面的代码报错了。

如果想要把b变为全局变量，要使用global声明。

In [8]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(1)

1
6


In [9]:
b  # b作为全局变量，已经被改变值

9

## 闭包

闭包指延伸了作用域的函数。其中包含函数体定义体重引用，但不在定义体重定义的非局部变量。

In [12]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [13]:
avg = Averager()
avg(10)

10.0

In [14]:
avg(12)

11.0

In [15]:
avg(20)

14.0

In [16]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [17]:
avg = make_averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

`make_averager`中，avg在哪存储历史值呢，是series，它被称作自由变量。指的是未在本地作用域中绑定的变量。

In [20]:
series

NameError: name 'series' is not defined

In [23]:
avg.__code__.co_varnames

('new_value', 'total')

In [24]:
avg.__code__.co_freevars

('series',)

`avg.__closure__` 中的各个元素对应于 `avg.__code__.co_freevars` 中的一个名称。

这些元素是cell对象，有个cell_content属性，保存着真正的值。

In [25]:
avg.__closure__

(<cell at 0x10f227078: list object at 0x10f17ef08>,)

In [27]:
avg.__closure__[0].cell_contents

[10, 11, 12]

综上，闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

## nonlocal 声明

更好的方式，每次只存储目前的总值和元素个数。

In [28]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [29]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

前文的series是可变的list，此处count和total是不可变类型。

如果尝试重新绑定的时候，会隐式地创建局部变量count，这样count就不再是自由变量，因此会报错;

考虑使用`nonlocal`进行声明

In [31]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [32]:
avg = make_averager()
avg(10)

10.0

### 实现一个简单的装饰器

```python
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ",".join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s)  -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked
```

In [34]:
import time
from clockdeco_demo import clock

In [35]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40,  'Calling snooze(.123)')
snooze(.123)

print('*' * 49, 'Calling factorial(6)')
print('6!=', factorial(6))

**************************************** Calling snooze(.123)
[0.12388254s] snooze(0.123)  -> None
************************************************* Calling factorial(6)
[0.00000075s] factorial(1)  -> 1
[0.00004753s] factorial(2)  -> 2
[0.00008174s] factorial(3)  -> 6
[0.00011485s] factorial(4)  -> 24
[0.00014701s] factorial(5)  -> 120
[0.00018104s] factorial(6)  -> 720
6!= 720


## 标准库中的装饰器

Python内置了三个用于装饰方法的函数: property,classmethod,staticmethod

### 使用functools.lru_cache 做备忘

`functools.lru_cache`是非常实用的装饰器，它实现了备忘功能。避免传入相同参数的时候重复计算，保留一部分最近使用的变量。

In [36]:
import functools

from clockdeco_demo import clock

In [37]:
@functools.lru_cache()
@clock
def fibnocci(n):
    if n < 2:
        return n
    return fibnocci(n-2) + fibnocci(n-1)

In [38]:
fibnocci(7)

[0.00000070s] fibnocci(1)  -> 1
[0.00000070s] fibnocci(0)  -> 0
[0.00002519s] fibnocci(2)  -> 1
[0.00012458s] fibnocci(3)  -> 2
[0.00000176s] fibnocci(4)  -> 3
[0.00016694s] fibnocci(5)  -> 5
[0.00000113s] fibnocci(6)  -> 8
[0.00021054s] fibnocci(7)  -> 13


13

In [39]:
fibnocci(30)

[0.00000199s] fibnocci(8)  -> 21
[0.00000137s] fibnocci(9)  -> 34
[0.00010708s] fibnocci(10)  -> 55
[0.00000135s] fibnocci(11)  -> 89
[0.00015433s] fibnocci(12)  -> 144
[0.00000121s] fibnocci(13)  -> 233
[0.00021359s] fibnocci(14)  -> 377
[0.00000130s] fibnocci(15)  -> 610
[0.00025736s] fibnocci(16)  -> 987
[0.00000128s] fibnocci(17)  -> 1597
[0.00030116s] fibnocci(18)  -> 2584
[0.00000112s] fibnocci(19)  -> 4181
[0.00034350s] fibnocci(20)  -> 6765
[0.00000101s] fibnocci(21)  -> 10946
[0.00038958s] fibnocci(22)  -> 17711
[0.00000106s] fibnocci(23)  -> 28657
[0.00043005s] fibnocci(24)  -> 46368
[0.00000102s] fibnocci(25)  -> 75025
[0.00046879s] fibnocci(26)  -> 121393
[0.00000112s] fibnocci(27)  -> 196418
[0.00050859s] fibnocci(28)  -> 317811
[0.00000093s] fibnocci(29)  -> 514229
[0.00055574s] fibnocci(30)  -> 832040


832040

可以配置的参数

`functools.lru_cache(maxsize=128, typed=False)`


### 单分派泛函数

`functools.singledispatch`装饰器可以把整体方案拆分成多个模块。将普通函数变成泛函数。

In [40]:
from functools import singledispatch
from collections import abc
import numbers
import html

In [41]:
@singledispatch
def htmlsize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlsize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlsize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlsize.register(tuple)
@htmlsize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlsize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n<ul>'

## 叠放装饰器

上述可以进行装饰器叠放


## 参数化装饰器

可以创建一个装饰器工厂函数，将一个参数传给他，产生对应的装饰器。

In [42]:
registry = []


def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x10f246b70>)
running main()
registry -> [<function f1 at 0x10f246b70>]
running f1()


### 一个参数化的注册装饰器


为了便于启用或者禁用函数注册功能，我们为它提供一个可选的active参数，设为False时，不注册被装饰的函数，而是装饰器工厂函数。

In [44]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'
             % (active, func))
        
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')


running register(active=False)->decorate(<function f1 at 0x10f21d620>)
running register(active=True)->decorate(<function f2 at 0x10f246268>)


In [45]:
registry

{<function __main__.f2>}

### 参数化clock装饰器

为它添加一个功能：当用户传入一个格式字符串，控制被装饰函数的输出

In [46]:
from clockdeco_param import clock

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i)

[0.00000596s] snooze(0) -> None
[1.00132298s] snooze(1) -> None
[2.00144506s] snooze(2) -> None


In [47]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i)

snooze: 5.9604644775390625e-06s
snooze: 1.0032501220703125s
snooze: 2.000133991241455s


In [48]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i)

snooze(0) dt=0.000s
snooze(1) dt=1.005s
snooze(2) dt=2.003s
